# `SVM` with `NB` features

This notebook implements shows how to use the NB-SVM classifier from [this paper](https://nlp.stanford.edu/pubs/sidaw12_simple_sentiment.pdf) that has become a classic (and competitive) baseline.

## toy dataset (binary)

Working example borrowed from [Stanford IR online text](https://nlp.stanford.edu/IR-book/html/htmledition/naive-bayes-text-classification-1.html)

| docID | text                                | label     |
|-------|-------------------------------------|-----------|
| 1     | chinese beijing chinese             | CHINA     |
| 2     | chinese chinese shanghai            | CHINA     |
| 3     | chinese macao                       | CHINA     |
| 4     | tokyo japan chinese                 | NOT_CHINA |
| 5     | chinese chinese chinese tokyo japan | ???       |


In [1]:
doc_one = "chinese beijing chinese"
label_one = 1

doc_two = "chinese chinese shanghai"
label_two = 1

doc_three = "chinese macao"
label_three = 1

doc_four = "tokyo japan chinese"
label_four = 0

all_docs = [doc_one, doc_two, doc_three, doc_four]
all_labels = [label_one, label_two, label_three, label_four]

In [2]:
from collections import OrderedDict
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import *
from sklearn.linear_model import *
from sklearn.naive_bayes import *
from sklearn.ensemble import *
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import accuracy_score
import numpy as np
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import inspect
import os
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0, os.path.join(parentdir, 'nb_transformer'))
from nb_transformer import NaiveBayesTransformer
from nb_enhanced_classifier import NaiveBayesEnhancedClassifier

## traditional `bernoulli` `vectorizer`

First let's `vectorize` the documents in the training set using `CountVectorizer()`. <br>
The paper found binary features to be more effective than raw counts.

In [3]:
vectorizer_bernoulli = CountVectorizer(
    binary=True
)
X_bernoulli = vectorizer_bernoulli.fit_transform(all_docs)

In [4]:
vectorizer_bernoulli.vocabulary_

{'chinese': 1, 'beijing': 0, 'shanghai': 4, 'macao': 3, 'tokyo': 5, 'japan': 2}

In [5]:
X_bernoulli.toarray()

array([[1, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 0, 1]], dtype=int64)

## `NaiveBayesTransformer`

This is a custom `transformer` that will use the Naive Bayes conditional probabilities as feature values.  See [the paper](https://nlp.stanford.edu/pubs/sidaw12_simple_sentiment.pdf) for details.

The key thing to recognize is that this method is built off of the assumption of a `binary` (only two labels) problem.  It can be expanded to `multiclass` utilizing the `one-v-all` approach, but this also requires a **different** transformation for **each** label, thus the inclusion of `label_of_interest` as an argument to the `NaiveBayesTransformer`.

In [6]:
nb_transformer = NaiveBayesTransformer(all_labels, 1)

### original feature space

In [7]:
original_features = vectorizer_bernoulli.fit_transform(all_docs)
original_features.toarray()

array([[1, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 0, 1]], dtype=int64)

### transformed feature space

In [8]:
nb_transformed = nb_transformer.fit_transform(original_features)
nb_transformed

array([[ 0.4054651 ,  0.4054651 , -0.        ,  0.        ,  0.        ,
        -0.        ],
       [ 0.        ,  0.4054651 , -0.        ,  0.        ,  0.4054651 ,
        -0.        ],
       [ 0.        ,  0.4054651 , -0.        ,  0.4054651 ,  0.        ,
        -0.        ],
       [ 0.        ,  0.4054651 , -0.98082924,  0.        ,  0.        ,
        -0.98082924]])

## Train

Given our features, let's train a classifier to predict `1` v. `0`. 
The paper found an `SVM` with `squared loss` and `l2` penalizing to be most effective.

In [9]:
svm_clf = LinearSVC(            
    loss='squared_hinge',
    penalty='l2',
    random_state = 1,         # to ensure reproducible results
    class_weight='balanced',
    dual=False
)

Let's build an instance of `NaiveBayesEnhancedClassifier`.

In [10]:
svm_clf.get_params()

{'C': 1.0,
 'class_weight': 'balanced',
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'loss': 'squared_hinge',
 'max_iter': 1000,
 'multi_class': 'ovr',
 'penalty': 'l2',
 'random_state': 1,
 'tol': 0.0001,
 'verbose': 0}

In [11]:
binary_example_clf = NaiveBayesEnhancedClassifier(
    base_clf=svm_clf
)

Let's train it on our dataset by feeding the *original* vectorized features into `.fit()`

In [12]:
binary_example_clf.fit(original_features, all_labels)

NaiveBayesEnhancedClassifier(base_clf=LinearSVC(C=1.0, class_weight='balanced', dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=1, tol=0.0001,
     verbose=0),
               interpolation_factor=0.25)

Baked into `.fit()` are the following steps:
 1. `transform` the features using the `NaiveBayesTransformer`
 2. Apply an interpolation of weights to the `.coef_` of the `classifier` using the formula below: 
  - $w' = (1 - \beta)\bar{w} + \beta w$ where $\bar{w}$ is the mean magnitude of all the weights

## Test

Now let's predict on a new data point.

In [13]:
test_doc = "chinese chinese chinese tokyo japan"

We first must `vectorize` the test item into the original feature space.

In [14]:
test_features_original = vectorizer_bernoulli.transform([test_doc])
test_features_original.toarray()

array([[0, 1, 1, 0, 0, 1]], dtype=int64)

Then we can use `NaiveBayesEnhancedClassifier.predict()` to (1) apply the `NB-transformation` and make our prediction.

In [15]:
binary_example_clf.predict(test_features_original)

array([0], dtype=int64)

We can see the "confidence" of this prediction using `NaiveBayesEnhancedClassifier.decision_function_predict_proba(test_features_original)` which will use `.decision_function()` or `.predict_proba()` of the original `classifier`.

In [16]:
binary_example_clf.decision_function_predict_proba(test_features_original)

array([-0.23337689])

The model predicts the test document to be in the `not China` class but not with much confidence.  

Note: This is opposite the prediction made in the Stanford IR worked example because they use `multinomial NB` and the presence of "China" three times outweighed the presence of the other words.

## `multiclass` test



As explained above, this transformation is built for `binary` classification.  The `multiclass` version requires the application of `one-v-all` to a traditional `SVM` classifier.  This is all done automatically within the `NaiveBayesEnhancedClassifier` `class`.  An example is below.

In [17]:
obvious_docs = [
    "cat",
    "cat",
    "cat",
    "pencil",
    "pencil",
    "pencil",
    "car",
    "car",
    "car",
]

obvious_labels = [
    0,
    0,
    0,
    1,
    1,
    1,
    2,
    2,
    2
]

test_docs = [
    "cat",
    "pen",
    "pencil",
    "car"
]

### traditional `bernoulli` `vectorizer`

In [18]:
multiclass_bernoulli_vectorizer = CountVectorizer(
    binary=True
)

multiclass_original_features = multiclass_bernoulli_vectorizer.fit_transform(obvious_docs)

In [19]:
multiclass_original_features.toarray()

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]], dtype=int64)

### Train

In [20]:
multiclass_example_clf = NaiveBayesEnhancedClassifier(
    # we'll use the default settings for `clf`
)

In [21]:
multiclass_example_clf.fit(
    multiclass_bernoulli_vectorizer.fit_transform(obvious_docs),   # vectorize the original documents
    obvious_labels
)

NaiveBayesEnhancedClassifier(base_clf=LinearSVC(C=1.0, class_weight='balanced', dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
               interpolation_factor=0.25)

### Test

Let's project the test documents into the same feature space as the training data.

In [22]:
multiclass_test_features = multiclass_bernoulli_vectorizer.transform(test_docs)
multiclass_test_features.toarray()

array([[0, 1, 0],
       [0, 0, 0],
       [0, 0, 1],
       [1, 0, 0]], dtype=int64)

Then let's get predictions for each test document.

In [23]:
multiclass_example_clf.predict(multiclass_test_features)

array([0, 0, 1, 2])

Behind the scenes, the classifier is generating predictions for how likely the data point is in that class.  We can see these values by calling `NaiveBayesEnhancedClassifier.decision_function_predict_proba()`.

Below, each row is a classifier trained for a different label.  The values in the vector represent the margin of each data point from the decision boundary.

In [24]:
for i, row in zip([0,1,2], multiclass_example_clf.decision_function_predict_proba(multiclass_test_features)):
    print(i, row)

0 [ 0.48603642 -0.4641514  -0.98429487 -1.06914084]
1 [-0.98429487 -0.4641514   0.48603642 -1.06914084]
2 [-0.98429487 -0.4641514  -0.98429487  0.64103137]


We then select the "most confident" classifier for each data point, where "most confident" is the one with the largest *positive* (or, if they're all *negative*, the *smallest*) margin.  In this case, each column is a data point, so we take the `argmax` of each column.

Note: Since the second data point ("pen") was never seen in the training vocabulary, the classifiers all generate the same prediction, none of which are positive (implying a low level of confidence)

### using `classifier` without `.coef_` or `.decision_function()`

The `NaiveBayesEnhancedClassifier` is built to wrap *any* `sklearn` classifier.  If the classifier does *not* use weights that are captured in the attribute, `.coef_`, then `interpolation` is skipped.

In [25]:
multiclass_example_no_decision_function = NaiveBayesEnhancedClassifier(
    base_clf=RandomForestClassifier()
)

In [26]:
multiclass_example_no_decision_function.fit(
    multiclass_bernoulli_vectorizer.fit_transform(obvious_docs),   # vectorize the original documents
    obvious_labels
)

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

C:\Users\michael.capizzi\Documents\projects\naive-bayes-transformer\nb_transformer\nb_enhanced_classifier.py:190: UserWarning:

the classifier you instantiated does not have an attribute `.coef_`; interpolation will not occur



NaiveBayesEnhancedClassifier(base_clf=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
               interpolation_factor=0.25)

Likewise, if the `classifier` chosen does *not* have a `.decision_function()` method, then `.predict_proba()` is used.

In [27]:
multiclass_example_no_decision_function.decision_function_predict_proba(multiclass_test_features)

C:\Users\michael.capizzi\Documents\projects\naive-bayes-transformer\nb_transformer\nb_enhanced_classifier.py:261: UserWarning:

classifier you instantiated does not have a method, `decision_function()`;using `predict_proba()` instead

C:\Users\michael.capizzi\Documents\projects\naive-bayes-transformer\nb_transformer\nb_enhanced_classifier.py:261: UserWarning:

classifier you instantiated does not have a method, `decision_function()`;using `predict_proba()` instead

C:\Users\michael.capizzi\Documents\projects\naive-bayes-transformer\nb_transformer\nb_enhanced_classifier.py:261: UserWarning:

classifier you instantiated does not have a method, `decision_function()`;using `predict_proba()` instead



array([[0.6, 0.6, 0. , 0.6],
       [0.1, 0.6, 0.6, 0.6],
       [0.1, 0.6, 0. , 1. ]])

In this case, each row still represents a `binary` classifier for a given label, and the vector values are the probabilities of the `positive` class.  We still choose the `argmax` in each column to determine our prediction.

In [28]:
multiclass_example_no_decision_function.predict(multiclass_test_features)

C:\Users\michael.capizzi\Documents\projects\naive-bayes-transformer\nb_transformer\nb_enhanced_classifier.py:261: UserWarning:

classifier you instantiated does not have a method, `decision_function()`;using `predict_proba()` instead

C:\Users\michael.capizzi\Documents\projects\naive-bayes-transformer\nb_transformer\nb_enhanced_classifier.py:261: UserWarning:

classifier you instantiated does not have a method, `decision_function()`;using `predict_proba()` instead

C:\Users\michael.capizzi\Documents\projects\naive-bayes-transformer\nb_transformer\nb_enhanced_classifier.py:261: UserWarning:

classifier you instantiated does not have a method, `decision_function()`;using `predict_proba()` instead



array([0, 0, 1, 2])

# Experiments on `20Newsgroups`

Below we attempt to recreate the `binary` classification problems in the experiments of the [original paper](https://nlp.stanford.edu/pubs/sidaw12_simple_sentiment.pdf).  Whenever provided, we use the same hyperparameters.

We also compare different classifiers and the use of (1) no transformer or (2) `TfidfVectorizer()`.

Note: It's likely that the data coming from `sklearn` for `20Newsgroups` is *not* exactly the same as the data used by the authors.  But the results are comparable.

## `alt.atheism` v. `talk.religion.misc` (`AthR`)

In [29]:
categories = [
    'alt.atheism', 'talk.religion.misc',
]

In [30]:
remove = (
    'headers'
)

## Data

In [31]:
data_train = fetch_20newsgroups(subset='train', categories=categories,
                                shuffle=True, random_state=42,
                                remove=remove)

data_test = fetch_20newsgroups(subset='test', categories=categories,
                               shuffle=True, random_state=42,
                               remove=remove)

idx2class_lookup = dict((i,c) for i,c in enumerate(data_train.target_names))
idx2class_lookup

{0: 'alt.atheism', 1: 'talk.religion.misc'}

In [32]:
# get label distribution
num_pos = list(data_train.target).count(1)
num_neg = list(data_train.target).count(0)
num_pos, num_neg

(377, 480)

### Vectorize

In [33]:
newsgroups_vectorizer_uni = CountVectorizer(
    binary=True
)

newsgroups_vectorizer_tfidf = TfidfVectorizer()

In [34]:
newsgroups_train_features_uni = newsgroups_vectorizer_uni.fit_transform(data_train.data)
newsgroups_train_features_uni

<857x17440 sparse matrix of type '<class 'numpy.int64'>'
	with 136539 stored elements in Compressed Sparse Row format>

In [35]:
newsgroups_test_features_uni = newsgroups_vectorizer_uni.transform(data_test.data)
newsgroups_test_features_uni

<570x17440 sparse matrix of type '<class 'numpy.int64'>'
	with 86232 stored elements in Compressed Sparse Row format>

### transformed - nb

In [36]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    random_state=1,
    dual=False
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

In [37]:
newsgroups_nb_plus_clfs = OrderedDict()
for clf_name, clf in all_classifiers.items():
    newsgroups_nb_plus_clfs[clf_name] = NaiveBayesEnhancedClassifier(
        clf,
        interpolation_factor=None
)

In [38]:
# transformed - nb
for clf_name, clf in newsgroups_nb_plus_clfs.items():
    print("training {} - nb transformed".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

training svm_from_paper - nb transformed
training sgd - nb transformed


C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.



training ridge - nb transformed
training perceptron - nb transformed


C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.



training passive_aggressive - nb transformed


C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.



training random_forest - nb transformed


C:\Users\michael.capizzi\Documents\projects\naive-bayes-transformer\nb_transformer\nb_enhanced_classifier.py:165: UserWarning:

the classifier you instantiated does not have an attribute `.coef_`; interpolation will not occur



training adaboost - nb transformed


C:\Users\michael.capizzi\Documents\projects\naive-bayes-transformer\nb_transformer\nb_enhanced_classifier.py:165: UserWarning:

the classifier you instantiated does not have an attribute `.coef_`; interpolation will not occur



In [39]:
newsgroups_nb_transformed_uni_results = OrderedDict()
for clf_name, clf in newsgroups_nb_plus_clfs.items():
    print("testing transformed {}".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_nb_transformed_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_nb_transformed_uni_results

testing transformed svm_from_paper
testing transformed sgd
testing transformed ridge
testing transformed perceptron
testing transformed passive_aggressive
testing transformed random_forest
testing transformed adaboost


OrderedDict([('svm_from_paper', 0.8333333333333334),
             ('sgd', 0.7964912280701755),
             ('ridge', 0.7912280701754386),
             ('perceptron', 0.7947368421052632),
             ('passive_aggressive', 0.8385964912280702),
             ('random_forest', 0.7789473684210526),
             ('adaboost', 0.7596491228070176)])

### no transformation

In [40]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    dual=False,
    C=0.1,
    random_state=1
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["bernoulli_nb"] = BernoulliNB(
    alpha=0.01,
)

all_classifiers["multinomial_nb"] = MultinomialNB(
    alpha=0.01,
)

In [41]:
# no transformation count
for clf_name, clf in all_classifiers.items():
    print("training baseline {} - no transformation".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

training baseline svm_from_paper - no transformation
training baseline sgd - no transformation


C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\ridge.py:332: UserWarning:

In Ridge, only 'sag' solver can currently fit the intercept when X is sparse. Solver has been automatically changed into 'sag'.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_

training baseline ridge - no transformation
training baseline perceptron - no transformation
training baseline passive_aggressive - no transformation
training baseline random_forest - no transformation
training baseline adaboost - no transformation
training baseline bernoulli_nb - no transformation
training baseline multinomial_nb - no transformation


In [42]:
newsgroups_no_transformation_uni_results = OrderedDict()
for clf_name, clf in all_classifiers.items():
    print("testing baseline {} - no transformation".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_no_transformation_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_no_transformation_uni_results

testing baseline svm_from_paper - no transformation
testing baseline sgd - no transformation
testing baseline ridge - no transformation
testing baseline perceptron - no transformation
testing baseline passive_aggressive - no transformation
testing baseline random_forest - no transformation
testing baseline adaboost - no transformation
testing baseline bernoulli_nb - no transformation
testing baseline multinomial_nb - no transformation


OrderedDict([('svm_from_paper', 0.7771929824561403),
             ('sgd', 0.756140350877193),
             ('ridge', 0.7701754385964912),
             ('perceptron', 0.7666666666666667),
             ('passive_aggressive', 0.7701754385964912),
             ('random_forest', 0.7789473684210526),
             ('adaboost', 0.7596491228070176),
             ('bernoulli_nb', 0.8315789473684211),
             ('multinomial_nb', 0.8368421052631579)])

### transformation - tfidf

In [43]:
newsgroups_train_features_uni = newsgroups_vectorizer_tfidf.fit_transform(data_train.data)
newsgroups_test_features_uni = newsgroups_vectorizer_tfidf.transform(data_test.data)

In [44]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    C=0.1,
    random_state=1,
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["bernoulli_nb"] = BernoulliNB(
    alpha=0.01
)

all_classifiers["multinomial_nb"] = MultinomialNB(
    alpha=0.01
)

In [45]:
for clf_name, clf in all_classifiers.items():
    print("training baseline {} - tfidf transformation".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

training baseline svm_from_paper - tfidf transformation
training baseline sgd - tfidf transformation
training baseline ridge - tfidf transformation
training baseline perceptron - tfidf transformation
training baseline passive_aggressive - tfidf transformation
training baseline random_forest - tfidf transformation


C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\ridge.py:332: UserWarning:

In Ridge, only 'sag' solver can currently fit the intercept when X is sparse. Solver has been automatically changed into 'sag'.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_

training baseline adaboost - tfidf transformation
training baseline bernoulli_nb - tfidf transformation
training baseline multinomial_nb - tfidf transformation


In [46]:
newsgroups_tfidf_transformed_uni_results = OrderedDict()
for clf_name, clf in all_classifiers.items():
    print("testing baseline {} - no transformation".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_tfidf_transformed_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_tfidf_transformed_uni_results

testing baseline svm_from_paper - no transformation
testing baseline sgd - no transformation
testing baseline ridge - no transformation
testing baseline perceptron - no transformation
testing baseline passive_aggressive - no transformation
testing baseline random_forest - no transformation
testing baseline adaboost - no transformation
testing baseline bernoulli_nb - no transformation
testing baseline multinomial_nb - no transformation


OrderedDict([('svm_from_paper', 0.7964912280701755),
             ('sgd', 0.8122807017543859),
             ('ridge', 0.8140350877192982),
             ('perceptron', 0.7929824561403509),
             ('passive_aggressive', 0.8210526315789474),
             ('random_forest', 0.7543859649122807),
             ('adaboost', 0.7421052631578947),
             ('bernoulli_nb', 0.8315789473684211),
             ('multinomial_nb', 0.8385964912280702)])

In [47]:
groups = [
    ("no_transform", newsgroups_no_transformation_uni_results),
    ("tfidf_transform", newsgroups_tfidf_transformed_uni_results), 
    ("nb_transform", newsgroups_nb_transformed_uni_results)
]

clfs = list(all_classifiers.keys())

traces = OrderedDict()
for i in range(len(groups)):
    transformation = groups[i][1]
    transformation_name = groups[i][0]
    scores = []
    for label in clfs:
        try:
            score = transformation[label]
        except:
            score = 0.0
        scores.append(score)
    traces["trace_{}".format(transformation_name)] = go.Bar(
        x=clfs,
        y=scores,
        name=transformation_name,
    )

data_ = [v for k, v in traces.items()]
layout_ = go.Layout(
    barmode='group',
    title='Transformation comparison: AthR (2.9)',
    yaxis=dict(
        range=[0, 1]
    )
)

fig_ = go.Figure(data=data_, layout=layout_)
iplot(fig_)    

## `comp.graphics` v. `comp.windows.x` (`XGraph`)

In [48]:
categories = [
    'comp.graphics', 'comp.windows.x',
]

## Data

In [49]:
data_train = fetch_20newsgroups(subset='train', categories=categories,
                                shuffle=True, random_state=42,
                                remove=remove)

data_test = fetch_20newsgroups(subset='test', categories=categories,
                               shuffle=True, random_state=42,
                               remove=remove)

idx2class_lookup = dict((i,c) for i,c in enumerate(data_train.target_names))
idx2class_lookup

{0: 'comp.graphics', 1: 'comp.windows.x'}

In [50]:
# get label distribution
num_pos = list(data_train.target).count(1)
num_neg = list(data_train.target).count(0)
num_pos, num_neg

(593, 584)

## Unigrams

### Vectorize

In [51]:
newsgroups_vectorizer_uni = CountVectorizer(
    binary=True
)

newsgroups_vectorizer_tfidf = TfidfVectorizer()

In [52]:
newsgroups_train_features_uni = newsgroups_vectorizer_uni.fit_transform(data_train.data)
newsgroups_train_features_uni

<1177x21163 sparse matrix of type '<class 'numpy.int64'>'
	with 136868 stored elements in Compressed Sparse Row format>

In [53]:
newsgroups_test_features_uni = newsgroups_vectorizer_uni.transform(data_test.data)
newsgroups_test_features_uni

<784x21163 sparse matrix of type '<class 'numpy.int64'>'
	with 89865 stored elements in Compressed Sparse Row format>

### transformed - nb

In [54]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    random_state=1,
    dual=False
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

In [55]:
list(map(lambda x: x[0], idx2class_lookup.items()))

[0, 1]

In [56]:
newsgroups_nb_plus_clfs = OrderedDict()
for clf_name, clf in all_classifiers.items():
    newsgroups_nb_plus_clfs[clf_name] = NaiveBayesEnhancedClassifier(
        clf,
        interpolation_factor=None
)

In [57]:
# transformed - nb
for clf_name, clf in newsgroups_nb_plus_clfs.items():
    print("training {} - nb transformed".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

training svm_from_paper - nb transformed
training sgd - nb transformed


C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.



training ridge - nb transformed
training perceptron - nb transformed


C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.



training passive_aggressive - nb transformed


C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.



training random_forest - nb transformed


C:\Users\michael.capizzi\Documents\projects\naive-bayes-transformer\nb_transformer\nb_enhanced_classifier.py:165: UserWarning:

the classifier you instantiated does not have an attribute `.coef_`; interpolation will not occur



training adaboost - nb transformed


C:\Users\michael.capizzi\Documents\projects\naive-bayes-transformer\nb_transformer\nb_enhanced_classifier.py:165: UserWarning:

the classifier you instantiated does not have an attribute `.coef_`; interpolation will not occur



In [58]:
newsgroups_nb_transformed_uni_results = OrderedDict()
for clf_name, clf in newsgroups_nb_plus_clfs.items():
    print("testing transformed {}".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_nb_transformed_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_nb_transformed_uni_results

testing transformed svm_from_paper
testing transformed sgd
testing transformed ridge
testing transformed perceptron
testing transformed passive_aggressive
testing transformed random_forest
testing transformed adaboost


OrderedDict([('svm_from_paper', 0.8329081632653061),
             ('sgd', 0.8367346938775511),
             ('ridge', 0.8214285714285714),
             ('perceptron', 0.8494897959183674),
             ('passive_aggressive', 0.8469387755102041),
             ('random_forest', 0.8061224489795918),
             ('adaboost', 0.8112244897959183)])

### no transformation

In [59]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    dual=False,
    C=0.1,
    random_state=1
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["bernoulli_nb"] = BernoulliNB(
    alpha=0.01,
)

all_classifiers["multinomial_nb"] = MultinomialNB(
    alpha=0.01,
)

In [60]:
# no transformation count
for clf_name, clf in all_classifiers.items():
    print("training baseline {} - no transformation".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

training baseline svm_from_paper - no transformation
training baseline sgd - no transformation
training baseline ridge - no transformation


C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\ridge.py:332: UserWarning:

In Ridge, only 'sag' solver can currently fit the intercept when X is sparse. Solver has been automatically changed into 'sag'.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_

training baseline perceptron - no transformation
training baseline passive_aggressive - no transformation
training baseline random_forest - no transformation
training baseline adaboost - no transformation
training baseline bernoulli_nb - no transformation
training baseline multinomial_nb - no transformation


In [61]:
newsgroups_no_transformation_uni_results = OrderedDict()
for clf_name, clf in all_classifiers.items():
    print("testing baseline {} - no transformation".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_no_transformation_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_no_transformation_uni_results

testing baseline svm_from_paper - no transformation
testing baseline sgd - no transformation
testing baseline ridge - no transformation
testing baseline perceptron - no transformation
testing baseline passive_aggressive - no transformation
testing baseline random_forest - no transformation
testing baseline adaboost - no transformation
testing baseline bernoulli_nb - no transformation
testing baseline multinomial_nb - no transformation


OrderedDict([('svm_from_paper', 0.8239795918367347),
             ('sgd', 0.8482142857142857),
             ('ridge', 0.8341836734693877),
             ('perceptron', 0.8329081632653061),
             ('passive_aggressive', 0.8290816326530612),
             ('random_forest', 0.8137755102040817),
             ('adaboost', 0.8112244897959183),
             ('bernoulli_nb', 0.8176020408163265),
             ('multinomial_nb', 0.8635204081632653)])

### transformation - tfidf

In [62]:
newsgroups_train_features_uni = newsgroups_vectorizer_tfidf.fit_transform(data_train.data)
newsgroups_test_features_uni = newsgroups_vectorizer_tfidf.transform(data_test.data)

In [63]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    C=0.1,
    random_state=1,
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["bernoulli_nb"] = BernoulliNB(
    alpha=0.01
)

all_classifiers["multinomial_nb"] = MultinomialNB(
    alpha=0.01
)

In [64]:
for clf_name, clf in all_classifiers.items():
    print("training baseline {} - tfidf transformation".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

training baseline svm_from_paper - tfidf transformation
training baseline sgd - tfidf transformation
training baseline ridge - tfidf transformation
training baseline perceptron - tfidf transformation
training baseline passive_aggressive - tfidf transformation
training baseline random_forest - tfidf transformation


C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\ridge.py:332: UserWarning:

In Ridge, only 'sag' solver can currently fit the intercept when X is sparse. Solver has been automatically changed into 'sag'.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_

training baseline adaboost - tfidf transformation
training baseline bernoulli_nb - tfidf transformation
training baseline multinomial_nb - tfidf transformation


In [65]:
newsgroups_tfidf_transformed_uni_results = OrderedDict()
for clf_name, clf in all_classifiers.items():
    print("testing baseline {} - no transformation".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_tfidf_transformed_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_tfidf_transformed_uni_results

testing baseline svm_from_paper - no transformation
testing baseline sgd - no transformation
testing baseline ridge - no transformation
testing baseline perceptron - no transformation
testing baseline passive_aggressive - no transformation
testing baseline random_forest - no transformation
testing baseline adaboost - no transformation
testing baseline bernoulli_nb - no transformation
testing baseline multinomial_nb - no transformation


OrderedDict([('svm_from_paper', 0.8482142857142857),
             ('sgd', 0.8571428571428571),
             ('ridge', 0.8545918367346939),
             ('perceptron', 0.8431122448979592),
             ('passive_aggressive', 0.8545918367346939),
             ('random_forest', 0.8086734693877551),
             ('adaboost', 0.7806122448979592),
             ('bernoulli_nb', 0.8176020408163265),
             ('multinomial_nb', 0.8673469387755102)])

In [66]:
groups = [
    ("no_transform", newsgroups_no_transformation_uni_results),
    ("tfidf_transform", newsgroups_tfidf_transformed_uni_results), 
    ("nb_transform", newsgroups_nb_transformed_uni_results)
]

clfs = list(all_classifiers.keys())

traces = OrderedDict()
for i in range(len(groups)):
    transformation = groups[i][1]
    transformation_name = groups[i][0]
    scores = []
    for label in clfs:
        try:
            score = transformation[label]
        except:
            score = 0.0
        scores.append(score)
    traces["trace_{}".format(transformation_name)] = go.Bar(
        x=clfs,
        y=scores,
        name=transformation_name,
    )

data_ = [v for k, v in traces.items()]
layout_ = go.Layout(
    barmode='group',
    title='Transformation comparison: XGraph (1.8)',
    yaxis=dict(
        range=[0, 1]
    )
)

fig_ = go.Figure(data=data_, layout=layout_)
iplot(fig_)    

## `rec.sport.baseball` v. `sci.crypt` (`BbCrypt`)

In [67]:
categories = [
    'rec.sport.baseball', 'sci.crypt',
]

## Data

In [68]:
data_train = fetch_20newsgroups(subset='train', categories=categories,
                                shuffle=True, random_state=42,
                                remove=remove)

data_test = fetch_20newsgroups(subset='test', categories=categories,
                               shuffle=True, random_state=42,
                               remove=remove)

idx2class_lookup = dict((i,c) for i,c in enumerate(data_train.target_names))
idx2class_lookup

{0: 'rec.sport.baseball', 1: 'sci.crypt'}

In [69]:
# get label distribution
num_pos = list(data_train.target).count(1)
num_neg = list(data_train.target).count(0)
num_pos, num_neg

(595, 597)

In [70]:
# determine intercept (log(N+/N-))
intercept = np.log(num_pos/num_neg)
intercept

-0.0033557078469723042

## Unigrams

### Vectorize

In [71]:
newsgroups_vectorizer_uni = CountVectorizer(
    binary=True
)

newsgroups_vectorizer_tfidf = TfidfVectorizer()

In [72]:
newsgroups_train_features_uni = newsgroups_vectorizer_uni.fit_transform(data_train.data)
newsgroups_train_features_uni

<1192x21197 sparse matrix of type '<class 'numpy.int64'>'
	with 176313 stored elements in Compressed Sparse Row format>

In [73]:
newsgroups_test_features_uni = newsgroups_vectorizer_uni.transform(data_test.data)
newsgroups_test_features_uni

<793x21197 sparse matrix of type '<class 'numpy.int64'>'
	with 97031 stored elements in Compressed Sparse Row format>

### transformed - nb

In [74]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    random_state=1,
    dual=False
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

In [75]:
newsgroups_nb_plus_clfs = OrderedDict()
for clf_name, clf in all_classifiers.items():
    newsgroups_nb_plus_clfs[clf_name] = NaiveBayesEnhancedClassifier(
        clf,
        interpolation_factor=None
)

In [76]:
# transformed - nb
for clf_name, clf in newsgroups_nb_plus_clfs.items():
    print("training {} - nb transformed".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

training svm_from_paper - nb transformed
training sgd - nb transformed


C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.



training ridge - nb transformed
training perceptron - nb transformed


C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.



training passive_aggressive - nb transformed


C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.



training random_forest - nb transformed


C:\Users\michael.capizzi\Documents\projects\naive-bayes-transformer\nb_transformer\nb_enhanced_classifier.py:165: UserWarning:

the classifier you instantiated does not have an attribute `.coef_`; interpolation will not occur



training adaboost - nb transformed


C:\Users\michael.capizzi\Documents\projects\naive-bayes-transformer\nb_transformer\nb_enhanced_classifier.py:165: UserWarning:

the classifier you instantiated does not have an attribute `.coef_`; interpolation will not occur



In [77]:
newsgroups_nb_transformed_uni_results = OrderedDict()
for clf_name, clf in newsgroups_nb_plus_clfs.items():
    print("testing transformed {}".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_nb_transformed_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_nb_transformed_uni_results

testing transformed svm_from_paper
testing transformed sgd
testing transformed ridge
testing transformed perceptron
testing transformed passive_aggressive
testing transformed random_forest
testing transformed adaboost


OrderedDict([('svm_from_paper', 0.9621689785624212),
             ('sgd', 0.9684741488020177),
             ('ridge', 0.9432534678436317),
             ('perceptron', 0.9810844892812106),
             ('passive_aggressive', 0.969735182849937),
             ('random_forest', 0.9432534678436317),
             ('adaboost', 0.9319041614123581)])

### no transformation

In [78]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    dual=False,
    C=0.1,
    random_state=1
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["bernoulli_nb"] = BernoulliNB(
    alpha=0.01,
)

all_classifiers["multinomial_nb"] = MultinomialNB(
    alpha=0.01,
)

In [79]:
# no transformation count
for clf_name, clf in all_classifiers.items():
    print("training baseline {} - no transformation".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

training baseline svm_from_paper - no transformation
training baseline sgd - no transformation
training baseline ridge - no transformation


C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\ridge.py:332: UserWarning:

In Ridge, only 'sag' solver can currently fit the intercept when X is sparse. Solver has been automatically changed into 'sag'.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.



training baseline perceptron - no transformation
training baseline passive_aggressive - no transformation
training baseline random_forest - no transformation


C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.



training baseline adaboost - no transformation
training baseline bernoulli_nb - no transformation
training baseline multinomial_nb - no transformation


In [80]:
newsgroups_no_transformation_uni_results = OrderedDict()
for clf_name, clf in all_classifiers.items():
    print("testing baseline {} - no transformation".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_no_transformation_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_no_transformation_uni_results

testing baseline svm_from_paper - no transformation
testing baseline sgd - no transformation
testing baseline ridge - no transformation
testing baseline perceptron - no transformation
testing baseline passive_aggressive - no transformation
testing baseline random_forest - no transformation
testing baseline adaboost - no transformation
testing baseline bernoulli_nb - no transformation
testing baseline multinomial_nb - no transformation


OrderedDict([('svm_from_paper', 0.9583858764186634),
             ('sgd', 0.9558638083228247),
             ('ridge', 0.9470365699873896),
             ('perceptron', 0.9520807061790668),
             ('passive_aggressive', 0.9596469104665826),
             ('random_forest', 0.9495586380832283),
             ('adaboost', 0.9319041614123581),
             ('bernoulli_nb', 0.9470365699873896),
             ('multinomial_nb', 0.9861286254728878)])

### transformation - tfidf

In [81]:
newsgroups_train_features_uni = newsgroups_vectorizer_tfidf.fit_transform(data_train.data)
newsgroups_test_features_uni = newsgroups_vectorizer_tfidf.transform(data_test.data)

In [82]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    C=0.1,
    random_state=1,
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["bernoulli_nb"] = BernoulliNB(
    alpha=0.01
)

all_classifiers["multinomial_nb"] = MultinomialNB(
    alpha=0.01
)

In [83]:
for clf_name, clf in all_classifiers.items():
    print("training baseline {} - tfidf transformation".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

training baseline svm_from_paper - tfidf transformation
training baseline sgd - tfidf transformation
training baseline ridge - tfidf transformation
training baseline perceptron - tfidf transformation
training baseline passive_aggressive - tfidf transformation
training baseline random_forest - tfidf transformation


C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\ridge.py:332: UserWarning:

In Ridge, only 'sag' solver can currently fit the intercept when X is sparse. Solver has been automatically changed into 'sag'.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning:

n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.

C:\Users\michael.capizzi\.conda\envs\test\lib\site-packages\sklearn\linear_

training baseline adaboost - tfidf transformation
training baseline bernoulli_nb - tfidf transformation
training baseline multinomial_nb - tfidf transformation


In [84]:
newsgroups_tfidf_transformed_uni_results = OrderedDict()
for clf_name, clf in all_classifiers.items():
    print("testing baseline {} - no transformation".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_tfidf_transformed_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_tfidf_transformed_uni_results

testing baseline svm_from_paper - no transformation
testing baseline sgd - no transformation
testing baseline ridge - no transformation
testing baseline perceptron - no transformation
testing baseline passive_aggressive - no transformation
testing baseline random_forest - no transformation
testing baseline adaboost - no transformation
testing baseline bernoulli_nb - no transformation
testing baseline multinomial_nb - no transformation


OrderedDict([('svm_from_paper', 0.9520807061790668),
             ('sgd', 0.9634300126103404),
             ('ridge', 0.9709962168978562),
             ('perceptron', 0.9558638083228247),
             ('passive_aggressive', 0.9747793190416141),
             ('random_forest', 0.9495586380832283),
             ('adaboost', 0.9129886506935687),
             ('bernoulli_nb', 0.9470365699873896),
             ('multinomial_nb', 0.9798234552332913)])

In [85]:
groups = [
    ("no_transform", newsgroups_no_transformation_uni_results),
    ("tfidf_transform", newsgroups_tfidf_transformed_uni_results), 
    ("nb_transform", newsgroups_nb_transformed_uni_results)
]

clfs = list(all_classifiers.keys())

traces = OrderedDict()
for i in range(len(groups)):
    transformation = groups[i][1]
    transformation_name = groups[i][0]
    scores = []
    for label in clfs:
        try:
            score = transformation[label]
        except:
            score = 0.0
        scores.append(score)
    traces["trace_{}".format(transformation_name)] = go.Bar(
        x=clfs,
        y=scores,
        name=transformation_name,
    )

data_ = [v for k, v in traces.items()]
layout_ = go.Layout(
    barmode='group',
    title='Transformation comparison: BbCrypt (0.5)',
    yaxis=dict(
        range=[0, 1]
    )
)

fig_ = go.Figure(data=data_, layout=layout_)
iplot(fig_)    

## `20Newsgroups`

In [86]:
categories = None

## Data

In [87]:
data_train = fetch_20newsgroups(subset='train', categories=categories,
                                shuffle=True, random_state=42,
                                remove=remove)

data_test = fetch_20newsgroups(subset='test', categories=categories,
                               shuffle=True, random_state=42,
                               remove=remove)

idx2class_lookup = dict((i,c) for i,c in enumerate(data_train.target_names))
idx2class_lookup

{0: 'alt.atheism',
 1: 'comp.graphics',
 2: 'comp.os.ms-windows.misc',
 3: 'comp.sys.ibm.pc.hardware',
 4: 'comp.sys.mac.hardware',
 5: 'comp.windows.x',
 6: 'misc.forsale',
 7: 'rec.autos',
 8: 'rec.motorcycles',
 9: 'rec.sport.baseball',
 10: 'rec.sport.hockey',
 11: 'sci.crypt',
 12: 'sci.electronics',
 13: 'sci.med',
 14: 'sci.space',
 15: 'soc.religion.christian',
 16: 'talk.politics.guns',
 17: 'talk.politics.mideast',
 18: 'talk.politics.misc',
 19: 'talk.religion.misc'}

## Unigrams

### Vectorize

In [88]:
newsgroups_vectorizer_uni = CountVectorizer(
    binary=True
)

newsgroups_vectorizer_tfidf = TfidfVectorizer()

In [89]:
newsgroups_train_features_uni = newsgroups_vectorizer_uni.fit_transform(data_train.data)
newsgroups_train_features_uni

<11314x125145 sparse matrix of type '<class 'numpy.int64'>'
	with 1604644 stored elements in Compressed Sparse Row format>

In [90]:
newsgroups_test_features_uni = newsgroups_vectorizer_uni.transform(data_test.data)
newsgroups_test_features_uni

<7532x125145 sparse matrix of type '<class 'numpy.int64'>'
	with 989304 stored elements in Compressed Sparse Row format>

### transformed - nb

In [91]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    random_state=1,
    dual=False
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

In [92]:
newsgroups_nb_plus_clfs = OrderedDict()
for clf_name, clf in all_classifiers.items():
    newsgroups_nb_plus_clfs[clf_name] = NaiveBayesEnhancedClassifier(
        clf,
        interpolation_factor=None
)

In [93]:
# transformed - nb
for clf_name, clf in newsgroups_nb_plus_clfs.items():
    print("training {} - nb transformed".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

training svm_from_paper - nb transformed


KeyboardInterrupt: 

In [ ]:
newsgroups_nb_transformed_uni_results = OrderedDict()
for clf_name, clf in newsgroups_nb_plus_clfs.items():
    print("testing transformed {}".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_nb_transformed_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_nb_transformed_uni_results

In [ ]:
newsgroups_nb_transformed_uni_results

### no transformation

In [ ]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    dual=False,
    C=0.1,
    random_state=1
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["bernoulli_nb"] = BernoulliNB(
    alpha=0.01,
)

all_classifiers["multinomial_nb"] = MultinomialNB(
    alpha=0.01,
)

In [ ]:
# no transformation count
for clf_name, clf in all_classifiers.items():
    print("training baseline {} - no transformation".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

In [ ]:
newsgroups_no_transformation_uni_results = OrderedDict()
for clf_name, clf in all_classifiers.items():
    print("testing baseline {} - no transformation".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_no_transformation_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_no_transformation_uni_results

### transformation - tfidf

In [ ]:
newsgroups_train_features_uni = newsgroups_vectorizer_tfidf.fit_transform(data_train.data)
newsgroups_test_features_uni = newsgroups_vectorizer_tfidf.transform(data_test.data)

In [ ]:
all_classifiers = OrderedDict()

all_classifiers["svm_from_paper"] = LinearSVC(
    loss='squared_hinge',
    penalty='l2',
    C=0.1,
    random_state=1,
)

all_classifiers["sgd"] = SGDClassifier(
    n_iter=50,
    penalty='elasticnet',
    random_state=1,
)

all_classifiers["ridge"] = RidgeClassifier(
    tol=1e-2,
    solver='lsqr',
    random_state=1,
)

all_classifiers["perceptron"] = Perceptron(
    n_iter=50,
    random_state=1,
)

all_classifiers["passive_aggressive"] = PassiveAggressiveClassifier(
    n_iter=50,
    random_state=1,
)

all_classifiers["random_forest"] = RandomForestClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["adaboost"] = AdaBoostClassifier(
    n_estimators=100,
    random_state=1,
)

all_classifiers["bernoulli_nb"] = BernoulliNB(
    alpha=0.01
)

all_classifiers["multinomial_nb"] = MultinomialNB(
    alpha=0.01
)

In [ ]:
for clf_name, clf in all_classifiers.items():
    print("training baseline {} - tfidf transformation".format(clf_name))
    clf.fit(newsgroups_train_features_uni, data_train.target)

In [ ]:
newsgroups_tfidf_transformed_uni_results = OrderedDict()
for clf_name, clf in all_classifiers.items():
    print("testing baseline {} - no transformation".format(clf_name))
    preds = clf.predict(newsgroups_test_features_uni)
    newsgroups_tfidf_transformed_uni_results[clf_name] = accuracy_score(data_test.target, preds)
newsgroups_tfidf_transformed_uni_results

In [ ]:
groups = [
    ("no_transform", newsgroups_no_transformation_uni_results),
    ("tfidf_transform", newsgroups_tfidf_transformed_uni_results), 
    ("nb_transform", newsgroups_nb_transformed_uni_results)
]

clfs = list(all_classifiers.keys())

traces = OrderedDict()
for i in range(len(groups)):
    transformation = groups[i][1]
    transformation_name = groups[i][0]
    scores = []
    for label in clfs:
        try:
            score = transformation[label]
        except:
            score = 0.0
        scores.append(score)
    traces["trace_{}".format(transformation_name)] = go.Bar(
        x=clfs,
        y=scores,
        name=transformation_name,
    )

data_ = [v for k, v in traces.items()]
layout_ = go.Layout(
    barmode='group',
    title='Transformation comparison: 20Newsgroups',
    yaxis=dict(
        range=[0, 1]
    )
)

fig_ = go.Figure(data=data_, layout=layout_)
iplot(fig_)    